# This is a note for [Camera Parameters - Extrinsics and Intrinsics (Cyrill Stachniss, 2020)](https://youtu.be/uHApDqH-8UE)

Personally, I prefer to setup the camera/image/sensor coordinate system as shown in the following [image](https://learnopencv.com/geometry-of-image-formation/). And I will use these coordinate system in this notebook.
![camera](./camera.png)

The ones used in the video are a bit confusing because there are two ways of forming the camera coordinate system in the video. One is at 6:20 and the image plane is behind the camera origin. The other one is at 23:19 and the z-axis is pointing away from the outside world. P.S. any of these two is good if you work out the math properly. But this is just a personal preference.

There are 4 coordinate system: world (3D) <-> camera (3D) -> image (2D) <-> sensor(2D).

Note: x_* are the euclidian coordinates, X_* are the homogeneous coordinates.

### world (3D) <-> camera (3D):
This is a revertible rigid body transform.

As a convention, the origin is at the pin hole, the image plane is in front of the pin hole instead of behind. This way, you do not need to flip the image. The z-axis is where the camera is pointing at. The x-axis and y-axis are pointing at the same direction of the x-axis and y-axis in the image coordinate system as well as the u and v in sensor coordinate system.

In [1]:
using NBInclude
@nbinclude("../Basics/rotation.ipynb")
x_world = rand(Uniform(0,1), (3))
X_world = HomogeneousCoord(EuclidianCoord(x_world)).coord
R = Rotation(AxisAngle(rand(Uniform(0,1), (3)))).matrix
t = rand(Uniform(0,1), (3))
f = rand() # unit: mm
ku, kv = rand(), rand() # unit: pixel/mm; this is the inverse of pixel spacing
cu, cv = 128 * rand(), 128 * rand() # unit: pixel;
s = rand(); # unit: pixel/mm

R is rotation matrix\
**t is the coordinate of world origin in camera coordinate system**\
**-R_T \* t is the coordinate of camera center (camera origin) in world coordinate system**

In [2]:
x_camera = R * x_world + t
X_camera = [R     t;
            0 0 0 1] * X_world
@assert HomogeneousCoord(EuclidianCoord(x_camera)) ≈ HomogeneousCoord(X_camera)
R_T = transpose(R)
O_center = - R_T * t
x_world_back = R_T * x_camera + O_center
@assert x_world == x_world

### camera (3D) -> image (2D):
This is an centeral projection.

In the camera coordinate system, there is a plane at z = f. This plane is the image coordinate system and also where the sensor is (Note that this means **z is by defination perpendicular to sensor**.). This is why its x and y axis have the same direction as camera coordinate system.

x_image = x_camera / z_camera * f\
y_image = y_camera / z_camera * f\
The unit of f is mm.

In [3]:
x_image = x_camera[1:2] * f / x_camera[3]
X_image = [f 0 0 0;
           0 f 0 0;
           0 0 1 0] * X_camera
@assert HomogeneousCoord(EuclidianCoord(x_image)) ≈ HomogeneousCoord(X_image)

### image (2D) <-> sensor(2D):
This is revertible transformation that may contain scale (x, y seperately), tranlation, skew.

Since there is no rotation, the u and v have the same direction as x and y axis in image/camera coordinate. As a convension, u is point horizontally to the right in the image, and v is pointing vertically down.

**[cu, cv] is the pixel coordinate of image origin (image center) in the sensor coordindate system**\
**The unit of cu and cv are pixel**\
The unit of ku, kv, and s are pixel/mm

In [4]:
x_sensor = [ku; kv] .* x_image + s * [x_image[2];0] + [cu;cv]
X_sensor = [ku s  cu;
            0  kv cv;
            0  0  1] * X_image
@assert HomogeneousCoord(EuclidianCoord(x_sensor)) ≈ HomogeneousCoord(X_sensor)

If we conbine the following two transformations, we get the intrinsic parameters:\
camera (3D) -> image (2D) <-> sensor(2D)

**The unit of ku\*f, kv\*f, and s\*f are also pixel**

In [5]:
K = [ku*f s*f  cu;
     0    kv*f cv;
     0    0    1]
X_sensor = [K zeros(3,1)] * X_camera
@assert HomogeneousCoord(EuclidianCoord(x_sensor)) ≈ HomogeneousCoord(X_sensor)

In [6]:
X_sensor = K * [R t] * X_world
@assert HomogeneousCoord(EuclidianCoord(x_sensor)) ≈ HomogeneousCoord(X_sensor)